In [28]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import datetime
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [1]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [2]:
print(y)
print('----------------------------------')
print(tX)
print('----------------------------------')
print(ids)

[ 1. -1. -1. ...  1. -1. -1.]
----------------------------------
[[ 138.47    51.655   97.827 ...    1.24    -2.475  113.497]
 [ 160.937   68.768  103.235 ... -999.    -999.      46.226]
 [-999.     162.172  125.953 ... -999.    -999.      44.251]
 ...
 [ 105.457   60.526   75.839 ... -999.    -999.      41.992]
 [  94.951   19.362   68.812 ... -999.    -999.       0.   ]
 [-999.      72.756   70.831 ... -999.    -999.       0.   ]]
----------------------------------
[100000 100001 100002 ... 349997 349998 349999]


In [99]:
# Not needed, why ?
# tx = np.insert(tX, 0, 1, axis=1)

## Cleaning the data
There seems to be quiet a few incorrect values "-999", for now we'll set these equal to the average of their respective column.

In [4]:
tx_ = np.where(tX==-999, np.nan, tX)
tx_

array([[138.47 ,  51.655,  97.827, ...,   1.24 ,  -2.475, 113.497],
       [160.937,  68.768, 103.235, ...,     nan,     nan,  46.226],
       [    nan, 162.172, 125.953, ...,     nan,     nan,  44.251],
       ...,
       [105.457,  60.526,  75.839, ...,     nan,     nan,  41.992],
       [ 94.951,  19.362,  68.812, ...,     nan,     nan,   0.   ],
       [    nan,  72.756,  70.831, ...,     nan,     nan,   0.   ]])

In [6]:
for i in range(0, 30):
    tX[:,i] = np.where(tX[:,i]==-999, np.nanmean(tx_[:,i]), tx_[:,i])

In [7]:
np.around(tX, 1)

array([[138.5,  51.7,  97.8, ...,   1.2,  -2.5, 113.5],
       [160.9,  68.8, 103.2, ...,  -0. ,  -0. ,  46.2],
       [121.9, 162.2, 126. , ...,  -0. ,  -0. ,  44.3],
       ...,
       [105.5,  60.5,  75.8, ...,  -0. ,  -0. ,  42. ],
       [ 95. ,  19.4,  68.8, ...,  -0. ,  -0. ,   0. ],
       [121.9,  72.8,  70.8, ...,  -0. ,  -0. ,   0. ]])

## Testing the functions
Everything goes into implementations.py.

In [17]:
%run implementations.py

In [35]:
weights_LS = least_squares(y, tX)[1]

In [24]:
weights_LSGD1 = least_squares_GD(y, tX, np.zeros(30), 100, 0.000001)[1]

# THIS CONVERGES INTO A LOCAL MINIMUM, solution: Dynamic Step Size ?

In [109]:
weights_LSGD2 = least_squares_GD(y, tX, [-1.15430619e+00,  1.82637143e-04, -7.20672107e-03, -6.45384640e-03,
        -1.73123839e-05,  2.32665195e-02,  4.20381986e-04,  2.50304362e-03,
         3.60203545e-01, -1.26385429e-03, -2.84568774e+00, -2.22719927e-01,
         9.89163555e-02,  3.56752661e-01,  2.85396180e+00, -6.42020723e-04,
        -4.57219107e-04,  2.85879539e+00, -6.80776737e-04,  1.38605239e-03,
         3.15125355e-03,  5.15272243e-04, -3.71558734e-04,  4.27220740e-02,
        -1.01225645e-03,  4.70620275e-04,  1.34341503e-04, -2.12423006e-03,
         1.42389540e-03, -1.78105047e-03,  2.84577828e+00], 100, 0.000001)[1]

# IT STAYS IN THE ABSOLUTE MINIMUM WHEN USING THAT AS A STARTING POINT

array([-1.15430619e+00,  1.82642973e-04, -7.20671861e-03, -6.45384287e-03,
       -1.73132964e-05,  2.32665198e-02,  4.20385733e-04,  2.50304348e-03,
        3.60203545e-01, -1.26385382e-03, -2.84568774e+00, -2.22719927e-01,
        9.89163555e-02,  3.56752661e-01,  2.85396180e+00, -6.42020722e-04,
       -4.57219112e-04,  2.85879539e+00, -6.80776738e-04,  1.38605239e-03,
        3.15125411e-03,  5.15272240e-04, -3.71557781e-04,  4.27220741e-02,
       -1.01225793e-03,  4.70620273e-04,  1.34341503e-04, -2.12423048e-03,
        1.42389540e-03, -1.78105047e-03,  2.84577828e+00])

In [26]:
weights_LSSGD = least_squares_SGD(y, tX, np.zeros(30), 100, 0.000001)[1]

In [32]:
weights_LSSGD

array([-5.58132234e-05, -7.94431272e-03, -5.92818185e-03, -3.57797529e-04,
       -2.49858267e-02,  1.10136033e-04, -2.17827311e-02,  2.80333385e-01,
       -9.86430949e-04,  6.77358458e-03, -2.34946194e-01,  9.85813757e-02,
        1.63447667e-01,  1.85388720e-03, -2.39497470e-03,  2.06968246e-04,
        6.76720733e-03, -3.31344346e-04,  1.40018108e-03,  2.55919430e-03,
        1.20974796e-03, -1.73282105e-04, -2.55260412e-01, -4.98412373e-03,
       -3.87776017e-04,  1.62296611e-03, -9.61199864e-03,  4.07822886e-03,
       -2.16323855e-03, -1.84235724e-03])

## Generate predictions and save ouput in csv format for submission:

In [21]:
DATA_TEST_PATH = 'test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [42]:
time_day = datetime.datetime.now().day
time_hour = datetime.datetime.now().hour
time_min = datetime.datetime.now().minute
time_second = datetime.datetime.now().second

time = str(time_day)+"-"+str(time_hour)+"-"+str(time_min)+"-"+str(time_second)

OUTPUT_PATH = 'submission'+"_"+str(time)+".csv"
print(weights_LS.shape)
y_pred = predict_labels(weights_LS, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

(30,)
